In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# 環境変数の取得
load_dotenv()

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [10]:
# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')
# データフレームを表示して確認
df.head()

,カテゴリー,商品コード,商品名,売上日,単価,数量,原価
0,食品,1001,りんご,2023-01-01,200,50,120
1,食品,1002,バナナ,2023-01-01,150,100,80
2,食品,1003,牛乳,2023-01-02,180,80,100
3,衣服,2001,Tシャツ,2023-01-02,1500,20,800
4,衣服,2002,ジーンズ,2023-01-03,5000,10,2500


In [11]:
# 2. データをLLM用にテキスト形式に変換
# データフレーム全体を文字列に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"
# 表示して確認
print(prompt_text)

売上データ:
    カテゴリー 商品コード      商品名         売上日    単価   数量    原価
0      食品  1001      りんご  2023-01-01   200   50   120
1      食品  1002      バナナ  2023-01-01   150  100    80
2      食品  1003       牛乳  2023-01-02   180   80   100
3      衣服  2001     Tシャツ  2023-01-02  1500   20   800
4      衣服  2002     ジーンズ  2023-01-03  5000   10  2500
..    ...   ...      ...         ...   ...  ...   ...
235    衣服  2077   レインパンツ  2023-04-28  2000   18  1000
236    食品  1085      ザクロ  2023-04-29   600   40   300
237   日用品  3077    バスブラシ  2023-04-29   400   60   200
238    衣服  2078  レインシューズ  2023-04-30  2500   15  1250
239    食品  1086    ココナッツ  2023-04-30   300   80   150

[240 rows x 7 columns]
この売上データの傾向を分析してください。


In [12]:
# 3. OpenAI APIの呼び出し

# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"

# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# LLMからの回答を表示
print(response.choices[0].message.content.strip())

売上データの傾向を分析するために、以下のポイントを考慮してみましょう。

### 1. 基本統計量
- **売上総額**：各商品の売上は「単価 × 数量」で計算されます。これにより、全体の売上を把握できます。
- **カテゴリー別売上**：食品、衣服、日用品などのカテゴリーごとに売上を集計し、どのカテゴリーが最も収益を上げているかを確認します。

### 2. 時間的な傾向
- **月別の売上**：売上日を基に月ごとの売上傾向を分析します。季節性が影響している場合、特定の月に売上が増加することが考えられます。
- **週別の売上**：同様に、週ごとの売上を分析することで、特定の曜日に売上が集中する傾向を探ることができます。

### 3. 商品別の売上
- **人気商品**：販売数量の観点から、最も売れている商品を特定します。これにより、今後のプロモーションや在庫管理に役立てることができます。
- **利益率分析**：各商品の利益（売上 - 原価）を計算し、利益率が高い商品を特定します。利益率の高い商品を重点的に販売促進することが可能です。

### 4. カテゴリーのパフォーマンス
- **カテゴリーごとの平均販売価格**：各カテゴリーごとに平均販売価格を算出し、価格戦略の見直しに役立てます。
- **在庫回転率**：各カテゴリーの売上と在庫量を踏まえて、在庫回転率を評価します。在庫が長期間動いていない商品を洗い出し、在庫管理の効率化を図ります。

### 5. その他のインサイト
- **プロモーションや季節の影響**：特定のプロモーションや季節的な要因（例えば、夏服の需要など）による売上の変動が見られるかどうかを分析します。

### 結論
分析を通じて、上記の傾向やインサイトを元に意思決定を行うことで、マーケティング戦略やプロモーション活動の最適化、在庫管理の改善、広告予算の配分などを行うことができます。

これらの観点を元にデータを深く掘り下げ、具体的な数字やトレンドを詳しく分析することで、企業の成長に寄与する洞察を生み出していきましょう。


In [5]:
# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])
print(df_out)

                                                   結果
0                 この売上データに基づいて、以下のような傾向やインサイトを分析できます。
1                                                    
2                                    ### 1. カテゴリー別の売上
3   - **食品、衣服、日用品**の3つの主要カテゴリーがあります。それぞれのカテゴリーがどれだ...
4                                                    
5                                   ### 2. 商品のパフォーマンス
6   - 商品コードや商品名ごとに売上と数量を集計し、最も売れている商品や利益が高い商品を特定しま...
7                                                    
8                                     ### 3. 価格と原価の分析
9   - 各商品の単価と原価を比較し、利益率を計算できます。利益率が高い商品に焦点を当てることで、...
10                                                   
11                                       ### 4. 時系列分析
12  - 売上日をもとに、月ごとの売上のトレンドを分析します。例えば、特定の月や季節に売上が増加す...
13                                                   
14                                        ### 5. 需要予測
15  - 過去のデータに基づいて、将来の需要を予測するためのモデルを構築することができます。これに...
16                                                   
17                          

In [7]:
# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

In [13]:
# ワークフロー化
print("処理を開始します。")

# 1. Excelファイルを読み込む
df = pd.read_excel('サンプルデータ.xlsx', sheet_name='売上データ')

# 2. データをLLM用にテキスト形式に変換
sales_data_text = df.astype(str)
prompt_text = f"売上データ:\n{sales_data_text}\nこの売上データの傾向を分析してください。"

# 3. OpenAI APIの呼び出し
# 役割を設定
role = "あなたはマーケティング分野に精通したデータサイエンティストです。企業の成長をサポートするために、効果的なインサイトを提供します。"
# APIへリクエスト
response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[
        {"role": "system", "content": role},
        {"role": "user", "content": prompt_text},
    ],
)

# 4. 分析結果をデータフレームに変換
result_list = response.choices[0].message.content.strip().split("\n")
df_out = pd.DataFrame(result_list, columns=['結果'])

# 5. 結果をExcelファイルに保存
df_out.to_excel("売上データ分析結果.xlsx", index=False)

print("Excelファイルに分析結果を保存しました。")

処理を開始します。
Excelファイルに分析結果を保存しました。
